In [ ]:
import numpy as np
import pandas as pd

import mysql.connector
from tqdm.notebook import tqdm


In [ ]:
session = mysql.connector.connect(
      host='localhost',
      port='3306',
      user='username',
      password='password',
      database='db_name'
  )

# コネクションが切れた時に再接続してくれるよう設定
session.ping(reconnect=True)

# 接続できているかどうか確認
print(session.is_connected())

# DB操作用にカーソルを作成
cur = session.cursor()


In [ ]:
#cur.execute("SELECT @@global.autocommit")


In [ ]:
cur.execute("SELECT * FROM horse_seiseki WHERE (year = 21) ORDER BY race_horse_id ASC limit 5")
rows = cur.fetchall()
for row in rows:
    print('data=', row)


In [ ]:
#cur.execute("SELECT * FROM horse_seiseki WHERE (blood_number = 2016104532) ORDER BY id ASC")
cur.execute("SELECT * FROM horse_seiseki WHERE (year = 21)")
# 全てのデータを取得
rows = cur.fetchall()
print(len(rows))

for row in rows:
    #print('data=', row)
    #print('data=', str(row[10])[:16])
    raceid = str(row[10])[:16]
    #print(f"UPDATE horse_seiseki SET race_id={raceid} WHERE id={row[0]}")
    cur.execute(f"UPDATE horse_seiseki SET race_id={raceid} WHERE id={row[0]}")


In [ ]:
path = './data/ZI/'
savefilename = 'ZI.csv' # 保存するファイル名、フルパスで指定


In [ ]:
df_zi_data = pd.read_csv(path + savefilename)
df_zi_data.columns = ['horse_id', 'ZI']
df_zi_data

In [ ]:
df_zi_data.query('horse_id==201501040601010102')['ZI'].values[0]

## 斤量差を求めてみる

In [ ]:
cur.execute("SELECT * FROM horse_seiseki WHERE (year >= 14 and year <= 21) ORDER BY race_id ASC")
#cur.execute("SELECT * FROM horse_seiseki WHERE (year = 21) ORDER BY race_id ASC")
rows = cur.fetchall()
#for row in rows:
#    print('data=', row)
cols =  ['id', '年', '月', '日', '日付S', '回次', '場所', '日次', '発走時刻', 'レース番号', 'レースID',
       '略レース名', 'クラスコード', '芝・ダ', 'トラックコード', 'コーナー回数', '距離', 'コース区分', '馬場状態',
       '天候', '馬名', '性別', '年齢', '騎手', '斤量', 'ブリンカー', '頭数', '枠番', '馬番', '入先着順',
       '異常コード', '確定着順', '着差タイム', '人気', '走破タイム秒', 'タイムS', '補正タイム', '補9',
       '通過順1角', '通過順2角', '通過順3角', '通過順4角', '脚質', '上がり3F', '馬体重', '増減', '調教師',
       '所属', '賞金', '血統登録番号', '騎手コード', '調教師コード', '馬主名', '生産者名', '父馬名', '母馬名',
       '母の父名', '毛色', '生年月日', '単勝オッズ', 'PCI', 'RPCI', 'PCI3', 'Ave-3F', 'レース名',
       '年齢限定競走種別コード', '競走記号コード', '重量コード', '1着本賞金', 'race_id', 'ZI', 'レース間隔_週', 'jocky_weight_diff', 'created_at', 'updated_at']
ddd = pd.DataFrame(rows, columns=cols)
ddd

In [ ]:
%%time

from datetime import datetime

# 競走馬の出走間隔を、週単位で算出する
def race_interval(arr):
    raceinterval = np.zeros(arr.shape[0])

    for i in range(arr.shape[0]):
        if i == 0:
            d = arr[i].split('.')
            d1 = datetime(int(d[0]), int(d[1]), int(d[2]))
            continue
        #print(i, arr[i])
        d = arr[i].split('.')
        d2 = datetime(int(d[0]), int(d[1]), int(d[2]))
        # レース間隔を取得
        dt = d2 - d1
        raceinterval[i] = dt.days // 7
         # 前の値を更新
        d1 = d2
    return raceinterval

bloodids = ddd['血統登録番号'].unique()
i = 0

for bloodid in bloodids:
    one_horse = ddd.query('血統登録番号 == @bloodid')[['レースID', '日付S', '斤量']]
    one_horse['斤量差'] = one_horse['斤量'].diff()
    one_horse['斤量差'] = one_horse['斤量差'].fillna(0.) # NaNも0.0に変換する
    
    one_horse['レース間隔week'] = race_interval(one_horse['日付S'].values)
    one_horse['レース間隔week'] = one_horse['レース間隔week'].astype(int)

    #print(one_horse)
    for race_unit in one_horse.itertuples():
        #if np.isnan(race_unit[4]):
        #    print(race_unit[4])
        #    race_unit[4] = 0. # Tupleは上書きできないのでエラー
        #print(race_unit[1], race_unit[4], race_unit[5])
        cur.execute(f"UPDATE horse_seiseki SET race_interval_week={race_unit[5]}, jocky_weight_diff={race_unit[4]} WHERE race_horse_id={race_unit[1]}")
    
    #if i > 0:
    #    break
    #i += 1

In [ ]:
cur.execute("COMMIT")

In [ ]:
%%time

#cur.execute("SELECT * FROM horse_seiseki WHERE (blood_number = 2016104532) ORDER BY id ASC")
cur.execute("SELECT * FROM horse_seiseki WHERE (year = 21)")
# 全てのデータを取得
rows = cur.fetchall()
print(len(rows))

zi_data = []
for row in tqdm(rows):
    try:
        #print('data=', row)
        #print('data=', str(row[10])[:16])
        raceid = row[10]
        zi = df_zi_data.query(f'horse_id=={raceid}')['ZI'].values[0].item()
        #print(f"UPDATE horse_seiseki SET race_id={raceid} WHERE id={row[0]}")
        #print(f"UPDATE horse_seiseki SET zi={zi} WHERE race_horse_id={raceid}")
        #print(type(zi), type(raceid))
        zi_data.append((zi, raceid)) # tupleで追加
    except Exception as e:
        #print(raceid, e)
        pass

if zi_data:
    print(f'-- コミット開始 {len(zi_data)} 個 --')
    #print(zi_data)
    sql = "UPDATE horse_seiseki SET zi = %s WHERE race_horse_id = %s"
    cur.executemany(sql, zi_data)
    cur.execute("COMMIT")
    print('-- コミット完了 --')
else:
    print(f'xx コミット失敗 {len(zi_data)} 個 xx')
    

In [ ]:
zi_data[0]

In [ ]:
cur.close()

In [ ]:
cur.execute("SELECT * FROM users ORDER BY id ASC")

# 全てのデータを取得
rows = cur.fetchall()
print(len(rows))


In [ ]:
for row in rows:
    print('data=', row)


In [ ]:
# executemanyで複数データを一度に挿入
records = [
  ('高橋よしのぶ', 43),
  ('阿部しんのすけ', 41),
]
cur.executemany("INSERT INTO users (`name`, `age`) VALUES (%s, %s)", records)



In [ ]:
cur.execute("COMMIT") # その他の接続でも見れるように(ROLLBACKが出来なくなる)

## 訓練データをデータベースへ登録

In [ ]:
horse_data = pd.read_csv('data/HorseSeiseki2008-2020+ZI.csv')
print(horse_data.columns, len(horse_data.columns)) # 71 col

In [ ]:
df_horse_data = horse_data.query('11<=年<=20').sort_values(['レースID'])
df_horse_data.info()

### データクレンジング

In [ ]:
def data_cleansing(horse_data):
    try:
        # ブリンカー
        #horse_data['ブリンカー'].unique() # array([nan, 'B'], dtype=object)
        horse_data['ブリンカー'].fillna(0, inplace=True) # 0    593765 付けてない
                                                          # B     50579
        # INTカラム -1 で埋める
        horse_data['コーナー回数'].fillna(-1, inplace=True)
        horse_data['補正タイム'].fillna(-1, inplace=True)
        horse_data['補9'].fillna(-1, inplace=True)
        horse_data['馬体重'].fillna(-1, inplace=True)
        horse_data['増減'].fillna(-999, inplace=True) # -999ありえない数値で埋めてみる
        horse_data['人気'].fillna(-999, inplace=True) # -999ありえない数値で埋めてみる

        # FLOATカラム -1.0で埋める
        horse_data['上がり3F'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['単勝オッズ'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['Ave-3F'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['PCI'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['RPCI'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['PCI3'].fillna(-1.0, inplace=True) # -1.0ありえない数値で埋めてみる
        horse_data['着差タイム'].replace('----', 99.9, inplace=True) # '----' 異常コード1以上の場合

        # charカラム スペース で埋める
        horse_data['コース区分'].fillna('', inplace=True)
        horse_data['脚質'].fillna('', inplace=True) # 脚質が定まっていない？

        # 例外の起こる可能性あり
        # INTカラム -1 で埋める
        horse_data['ZI'].fillna(-1, inplace=True)
    except KeyError as e:
        print('KeyError', e)
    except Exception as e:
        print(e)

In [ ]:
data_cleansing(df_horse_data)

In [ ]:
#df_horse_data['レースID'].map(lambda x: str(rid)[:16])


In [ ]:
df_horse_data['血統登録番号'].unique()

In [ ]:
df_horse_data[df_horse_data['馬名']=='グランアレグリア'][['略レース名', '入先着順', '騎手', '馬番', '上がり3F', 'Ave-3F', '馬体重', '増減', '調教師', '所属', '賞金', '血統登録番号']]

In [ ]:
#df_horse_data['異常コード'].value_counts()
df_horse_data[df_horse_data['着差タイム']=='----'][['異常コード', '場所', '芝・ダ', '距離', '略レース名', '走破タイム秒', '着差タイム']]

In [ ]:
df_horse_data[df_horse_data['着差タイム'].isnull()][['場所', '芝・ダ', '距離', '略レース名', '走破タイム秒', '着差タイム']]

### 型変換

In [ ]:
cols = ['年', '月', '日', '回次', 'レース番号', 'レースID', 'クラスコード', 'トラックコード', 'コーナー回数', '距離', '年齢', '頭数', '枠番', '馬番', '入先着順', '異常コード', '確定着順', \
        '人気', '補正タイム', '補9', '通過順1角', '通過順2角', '通過順3角', '通過順4角', '馬体重', '増減', '賞金', '血統登録番号', '騎手コード', '調教師コード', '生年月日', \
        '年齢限定競走種別コード', '重量コード', '1着本賞金', 'race_id', 'ZI', 'レース間隔_週']

try:
    for col in cols:
        df_horse_data[col] = df_horse_data[col].astype(int)
except Exception as e:
    print(e, '-->', col)

In [ ]:
df_horse_data.isnull().sum().tail()

In [ ]:
df_horse_data.info()

In [ ]:
df_horse_data.tail()

In [ ]:
df_horse_data[['人気', '走破タイム秒', 'タイムS', '補正タイム', '補9', 'ZI']]

In [ ]:
#type(result[0][0])

In [ ]:
# 馬ごとの成績データをデータベースへ登録する関数

def insert_table_all_train_data(horse_seiseki):
    try:
        years = horse_seiseki['年'].unique()
        for year in years:
            print('-', year, '年 ---')
            one_sycle = horse_seiseki.query('年==@year')
            horses = one_sycle.values.tolist()

            cur.executemany("INSERT INTO horse_seiseki (`year`, `month`, `day`, `date_short`, `kaiji`, \
                                                        `venue`, `nichiji`, `starts_at`, `race_no`, `race_horse_id`, \
                                                        `short_race_name`, `class_code`, `tarf_or_dart`, `track_code`, `times_corner`, \
                                                        `distance`, `cource_classification`, `venue_status`, `weather`, `horse_name`, \
                                                        `horse_sex`, `horse_age`, `jocky_name`, `jocky_weight`, `horse_gear`, \
                                                        `number_count`, `waku_number`, `uma_number`, `target_rank`, `abnormality_code`, \
                                                        `fixed_target_rank`, `time_diff`, `polularity`, `running_time_sec`, `running_time`, \
                                                        `adjustment_time`, `adjustment_time9`, `corner_rank1`, `corner_rank2`, `corner_rank3`, \
                                                        `corner_rank4`, `leg_type`, `last3_f`, `horse_weight`, `horse_weight_diff`, \
                                                        `trainer_name`, `home_ground`, `race_prize`, `blood_number`, `jocky_code`, \
                                                        `trainer_code`, `owner_name`, `producer_name`, `fathers_name`, `mothers_name`, \
                                                        `ground_father_name`, `fur_color`, `horse_birthday`, `tansho_odds`, `pci`, \
                                                        `rpci`, `pci3`, `ave3f`, `race_name`, `race_kind_code`, \
                                                        `race_mark_code`, `weight_code`, `real_prize`, `race_id`, `zi`, `race_interval_week` \
                                                       ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", horses)
        # 1ループ（1年単位）で反映
        cur.execute("COMMIT")
    except Exception as e:
        print(e)
    else:
        print('- 完了')

In [ ]:
%%time

insert_table_all_train_data(df_horse_data)

In [ ]:
horses = df_horse_data.values.tolist()

In [ ]:
%%time
'''
#for i, (name, soundrate, playtime, codec, waterproof) in enumerate(zip(names, rates, plytms, codecs, wtrprfs),1):
#    id = (i, name, soundrate, playtime, codec, waterproof)
#    horses.append(id)
    
records = df_horse_data.to_records(index=False)
#print(type(records)) #'numpy.recarray'
result = list(records)
#print(type(result[0])) #'numpy.record'
horses = [tuple(x) for x in result]
#print(result[0:2]), len(result)
#horses[0]
'''

In [ ]:
type(horses[0][0])

In [ ]:
#type(int(horses[0][2]))
records = [
  (int(horses[0][0]), int(horses[0][1]), int(horses[0][2]), horses[0][3], int(horses[0][4])),
  (int(horses[1][0]), int(horses[1][1]), int(horses[1][2]), horses[1][3], int(horses[1][4])),
]


In [ ]:
%%time

#cur.executemany("INSERT INTO horse_seiseki (`year`, `month`, `day`, `date_short`, `kaiji`) \
#                                          VALUES (%s, %s, %s, %s, %s)", records)


cur.executemany("INSERT INTO horse_seiseki (`year`, `month`, `day`, `date_short`, `kaiji`, \
                                            `venue`, `nichiji`, `starts_at`, `race_no`, `race_horse_id`, \
                                            `short_race_name`, `class_code`, `tarf_or_dart`, `track_code`, `times_corner`, \
                                            `distance`, `cource_classification`, `venue_status`, `weather`, `horse_name`, \
                                            `horse_sex`, `horse_age`, `jocky_name`, `jocky_weight`, `horse_gear`, \
                                            `number_count`, `waku_number`, `uma_number`, `target_rank`, `abnormality_code`, \
                                            `fixed_target_rank`, `time_diff`, `polularity`, `running_time_sec`, `running_time`, \
                                            `adjustment_time`, `adjustment_time9`, `corner_rank1`, `corner_rank2`, `corner_rank3`, \
                                            `corner_rank4`, `leg_type`, `last3_f`, `horse_weight`, `horse_weight_diff`, \
                                            `trainer_name`, `home_ground`, `race_prize`, `blood_number`, `jocky_code`, \
                                            `trainer_code`, `owner_name`, `producer_name`, `fathers_name`, `mothers_name`, \
                                            `ground_father_name`, `fur_color`, `horse_birthday`, `tansho_odds`, `pci`, \
                                            `rpci`, `pci3`, `ave3f`, `race_name`, `race_kind_code`, \
                                            `race_mark_code`, `weight_code`, `real_prize`, `race_id`, `zi`, `race_interval_week` \
                                           ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", horses)



In [ ]:
cur.execute("SELECT * FROM horse_seiseki ORDER BY id ASC")
# 全てのデータを取得
rows = cur.fetchall()
print(len(rows))


In [ ]:
# カラムの取得
cur.execute("DESC horse_seiseki")
rows = cur.fetchall()
print(type(rows), rows[5])
#for row in rows:
#    print(row[0])

In [ ]:
cur.execute("COMMIT")

In [ ]:
# 【注意】テーブルの初期化
cur.execute("TRUNCATE TABLE horse_seiseki")

In [ ]:
race_data = pd.read_csv('data/HorseSeiseki2021.csv')
cols = ['年',
        '月',
        '日',
        '日付S',
        '回次',
        '場所',
        '日次',
        '発走時刻',
        'レース番号',
        'レースID',
        '略レース名',
        'クラスコード',
        '芝・ダ',
        'トラックコード',
        'コーナー回数',
        '距離',
        'コース区分',
        '馬場状態',
        '天候',
        '馬名',
        '性別',
        '年齢',
        '騎手',
        '斤量',
        'ブリンカー',
        '頭数',
        '枠番',
        '馬番',
        '入先着順',
        '異常コード',
        '確定着順',
        '着差タイム',
        '人気',
        '走破タイム秒',
        'タイムS',
        '補正タイム',
        '補9',
        '通過順1角',
        '通過順2角',
        '通過順3角',
        '通過順4角',
        '脚質',
        '上がり3F',
        '馬体重',
        '増減',
        '調教師',
        '所属',
        '賞金',
        '血統登録番号',
        '騎手コード',
        '調教師コード',
        '馬主名',
        '生産者名',
        '父馬名',
        '母馬名',
        '母の父名',
        '毛色',
        '生年月日',
        '単勝オッズ',
        'PCI',
        'RPCI',
        'PCI3',
        'Ave-3F',
        'レース名',
        '年齢限定競走種別コード',
        '競走記号コード',
        '重量コード',
        '1着本賞金',
       ]
race_data.columns = cols


In [ ]:
race_data['race_id'] = race_data['レースID'].map(lambda x: str(x)[:16]).astype(int) # レースごとのIDを作る

In [ ]:
path = './data/ZI/'
savefilename = 'ZI.csv' # 保存するファイル名、フルパスで指定

df_zi_data = pd.read_csv(path + savefilename)
df_zi_data.columns = ['horse_id', 'ZI']
race_data = pd.merge(race_data, df_zi_data, left_on='レースID', right_on='horse_id', how='left').drop(columns='horse_id')
#race_data['ZI'] = mg['ZI'].fillna(0).astype(int) # データが入っていない項目が3947こ
race_data

In [ ]:
data_cleansing(race_data)
race_data

In [ ]:
# 型変換
cols = ['年', '月', '日', '回次', 'レース番号', 'レースID', 'クラスコード', 'トラックコード', 'コーナー回数', '距離', '年齢', '頭数', '枠番', '馬番', '入先着順', '異常コード', '確定着順', \
        '人気', '補正タイム', '補9', '通過順1角', '通過順2角', '通過順3角', '通過順4角', '馬体重', '増減', '賞金', '血統登録番号', '騎手コード', '調教師コード', '生年月日', \
        '年齢限定競走種別コード', '重量コード', '1着本賞金', 'race_id', 'ZI']

try:
    for col in cols:
        race_data[col] = race_data[col].astype(int)
except Exception as e:
    print(e, '-->', col)

In [ ]:
# 馬ごとの成績データをデータベースへ登録する関数

def insert_table_a_year_train_data(horse_seiseki):
    try:
        years = horse_seiseki['年'].unique()
        for year in years:
            print('-', year, '年 ---')
            one_sycle = horse_seiseki.query('年==@year')
            horses = one_sycle.values.tolist()

            cur.executemany("INSERT INTO horse_seiseki (`year`, `month`, `day`, `date_short`, `kaiji`, \
                                                        `venue`, `nichiji`, `starts_at`, `race_no`, `race_horse_id`, \
                                                        `short_race_name`, `class_code`, `tarf_or_dart`, `track_code`, `times_corner`, \
                                                        `distance`, `cource_classification`, `venue_status`, `weather`, `horse_name`, \
                                                        `horse_sex`, `horse_age`, `jocky_name`, `jocky_weight`, `horse_gear`, \
                                                        `number_count`, `waku_number`, `uma_number`, `target_rank`, `abnormality_code`, \
                                                        `fixed_target_rank`, `time_diff`, `polularity`, `running_time_sec`, `running_time`, \
                                                        `adjustment_time`, `adjustment_time9`, `corner_rank1`, `corner_rank2`, `corner_rank3`, \
                                                        `corner_rank4`, `leg_type`, `last3_f`, `horse_weight`, `horse_weight_diff`, \
                                                        `trainer_name`, `home_ground`, `race_prize`, `blood_number`, `jocky_code`, \
                                                        `trainer_code`, `owner_name`, `producer_name`, `fathers_name`, `mothers_name`, \
                                                        `ground_father_name`, `fur_color`, `horse_birthday`, `tansho_odds`, `pci`, \
                                                        `rpci`, `pci3`, `ave3f`, `race_name`, `race_kind_code`, \
                                                        `race_mark_code`, `weight_code`, `real_prize`, `race_id`, `zi` \
                                                       ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", horses)
        # 1ループ（1年単位）で反映
        cur.execute("COMMIT")
    except Exception as e:
        print('ERROR ', e)
    else:
        print('- 完了')

In [ ]:
insert_table_a_year_train_data(race_data)
